# CI/CD 자동화 실습: GitHub Actions & Azure Pipelines

> **실습 환경**: 이 노트북은 GitHub Codespaces에서 실행하도록 구성되어 있습니다.

> **사전 준비**: GitHub 저장소, Azure 구독, ACR, AKS 클러스터가 필요합니다.

이 노트북에서는 GitHub Actions와 Azure Pipelines를 활용하여 컨테이너 기반 애플리케이션의 CI/CD 자동화 실습을 진행합니다.

## 1. CI/CD란?
- **CI(Continuous Integration)**: 코드 변경 사항을 자동으로 빌드 및 테스트
- **CD(Continuous Delivery/Deployment)**: 애플리케이션을 자동으로 배포

### CI/CD의 이점
- 빠른 피드백
- 자동화된 품질 보증
- 신속한 배포

## 2. GitHub Actions로 CI/CD 구현

GitHub Actions는 GitHub 저장소에서 직접 워크플로우를 자동화할 수 있는 도구입니다.

### 2.1 GitHub Actions 워크플로우 구성

GitHub Codespaces에서 `.github/workflows` 디렉터리를 생성하고 워크플로우 파일을 작성합니다.

#### 디렉터리 생성

In [2]:
import subprocess
import os

# GitHub Actions 워크플로우 디렉터리 생성
workflow_dir = ".github/workflows"

if not os.path.exists(workflow_dir):
    os.makedirs(workflow_dir)
    print(f"✅ 디렉터리 생성 완료: {workflow_dir}")
else:
    print(f"✅ 디렉터리가 이미 존재합니다: {workflow_dir}")

# 디렉터리 구조 확인
subprocess.run(["ls", "-la", ".github/"])

✅ 디렉터리 생성 완료: .github/workflows
total 0
drwxr-xr-x@  3 junwoojeong  staff   96 Oct 11 20:50 .
drwxr-xr-x  11 junwoojeong  staff  352 Oct 11 20:50 ..
drwxr-xr-x@  2 junwoojeong  staff   64 Oct 11 20:50 workflows


CompletedProcess(args=['ls', '-la', '.github/'], returncode=0)

#### 워크플로우 파일 작성

`.github/workflows/build-and-push.yml` 파일을 생성하고 아래 내용을 작성하세요.

> **주의**: GitHub Secrets에 `AZURE_CREDENTIALS`, `ACR_USERNAME`, `ACR_PASSWORD`를 설정해야 합니다.

In [3]:
import os

# .github/workflows/build-and-push.yml 파일 생성
workflow_content = """name: Build and Push to ACR

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

jobs:
  build-and-push:
    runs-on: ubuntu-latest
    
    steps:
      - name: Checkout code
        uses: actions/checkout@v3
      
      - name: Set up JDK 17
        uses: actions/setup-java@v3
        with:
          java-version: '17'
          distribution: 'temurin'
      
      - name: Build with Maven
        run: |
          cd springboot-docker-demo
          mvn clean package
      
      - name: Azure Login
        uses: azure/login@v1
        with:
          creds: ${{ secrets.AZURE_CREDENTIALS }}
      
      - name: Login to ACR
        run: |
          az acr login --name ${{ secrets.ACR_NAME }}
      
      - name: Build and Push Docker Image
        run: |
          cd springboot-docker-demo
          docker build -t ${{ secrets.ACR_NAME }}.azurecr.io/myapp:${{ github.sha }} .
          docker build -t ${{ secrets.ACR_NAME }}.azurecr.io/myapp:latest .
          docker push ${{ secrets.ACR_NAME }}.azurecr.io/myapp:${{ github.sha }}
          docker push ${{ secrets.ACR_NAME }}.azurecr.io/myapp:latest
"""

# 파일 저장
workflow_file = ".github/workflows/build-and-push.yml"
os.makedirs(os.path.dirname(workflow_file), exist_ok=True)

with open(workflow_file, "w") as f:
    f.write(workflow_content)

print(f"✅ GitHub Actions 워크플로우 파일 생성 완료: {workflow_file}")
print()
print("📄 생성된 파일 내용:")
with open(workflow_file, "r") as f:
    print(f.read())

✅ GitHub Actions 워크플로우 파일 생성 완료: .github/workflows/build-and-push.yml

📄 생성된 파일 내용:
name: Build and Push to ACR

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

jobs:
  build-and-push:
    runs-on: ubuntu-latest

    steps:
      - name: Checkout code
        uses: actions/checkout@v3

      - name: Set up JDK 17
        uses: actions/setup-java@v3
        with:
          java-version: '17'
          distribution: 'temurin'

      - name: Build with Maven
        run: |
          cd springboot-docker-demo
          mvn clean package

      - name: Azure Login
        uses: azure/login@v1
        with:
          creds: ${{ secrets.AZURE_CREDENTIALS }}

      - name: Login to ACR
        run: |
          az acr login --name ${{ secrets.ACR_NAME }}

      - name: Build and Push Docker Image
        run: |
          cd springboot-docker-demo
          docker build -t ${{ secrets.ACR_NAME }}.azurecr.io/myapp:${{ github.sha }} .
          docker build -t ${{ secret

### 2.2 GitHub Secrets 설정

GitHub 저장소의 Settings > Secrets and variables > Actions에서 다음 Secrets를 추가하세요:

1. **AZURE_CREDENTIALS**: Azure Service Principal 자격 증명
2. **ACR_NAME**: Azure Container Registry 이름

#### Azure Service Principal 생성

In [4]:
import subprocess

print("🔑 Azure Service Principal 생성")
print("=" * 80)
print()
print("⚠️  아래 명령을 참고하여 Azure Portal 또는 Azure CLI에서 실행하세요:")
print()
print("az ad sp create-for-rbac --name 'github-actions-sp' --role contributor \\")
print("  --scopes /subscriptions/{subscription-id}/resourceGroups/{resource-group} \\")
print("  --sdk-auth")
print()
print("💡 참고:")
print("   1. {subscription-id}를 실제 구독 ID로 변경")
print("   2. {resource-group}을 실제 리소스 그룹 이름으로 변경")
print("   3. 출력된 JSON을 복사하여 GitHub Secrets의 AZURE_CREDENTIALS에 저장")
print()
print("=" * 80)

🔑 Azure Service Principal 생성

⚠️  아래 명령을 참고하여 Azure Portal 또는 Azure CLI에서 실행하세요:

az ad sp create-for-rbac --name 'github-actions-sp' --role contributor \
  --scopes /subscriptions/{subscription-id}/resourceGroups/{resource-group} \
  --sdk-auth

💡 참고:
   1. {subscription-id}를 실제 구독 ID로 변경
   2. {resource-group}을 실제 리소스 그룹 이름으로 변경
   3. 출력된 JSON을 복사하여 GitHub Secrets의 AZURE_CREDENTIALS에 저장



### 2.3 워크플로우 실행 및 모니터링

코드를 GitHub에 푸시하면 워크플로우가 자동으로 실행됩니다.

1. GitHub Codespaces에서 코드 변경
2. Git commit & push
3. GitHub 저장소의 Actions 탭에서 진행 상황 확인

In [ ]:
import subprocess

print("📤 Git 설정 및 코드 푸시")
print("=" * 80)
print()

# Git 상태 확인
print("1️⃣ Git 상태 확인:")
subprocess.run(["git", "status"])

print()
print("2️⃣ 변경 사항 추가:")
subprocess.run(["git", "add", ".github/workflows/"])

print()
print("3️⃣ 커밋:")
result = subprocess.run([
    "git", "commit", "-m", "Add GitHub Actions workflow"
], capture_output=True, text=True)

if result.returncode == 0:
    print("✅ 커밋 완료")
else:
    print("⚠️  커밋 결과:")
    print(result.stdout)
    print(result.stderr)

print()
print("=" * 80)
print("💡 다음 단계:")
print("   1. git push origin main 실행하여 코드 푸시")
print("   2. GitHub 저장소의 Actions 탭에서 워크플로우 실행 확인")
print("   3. URL: https://github.com/{username}/{repo}/actions")
print("=" * 80)

## 3. Azure Pipelines로 CI/CD 구현
Azure Pipelines는 다양한 플랫폼에서 빌드, 테스트, 배포를 자동화할 수 있는 Azure DevOps의 서비스입니다.

### 예시 파이프라인 (azure-pipelines.yml)
아래는 컨테이너 이미지를 빌드하고 ACR로 푸시하는 Azure Pipelines 예시입니다.

In [ ]:
import os

# azure-pipelines.yml 파일 생성
pipeline_content = """trigger:
- main

pool:
  vmImage: 'ubuntu-latest'

steps:
- checkout: self
- task: AzureCLI@2
  inputs:
    azureSubscription: 'AzureServiceConnection'
    scriptType: 'bash'
    scriptLocation: 'inlineScript'
    inlineScript: |
      az acr login --name myacr
      docker build -t myacr.azurecr.io/myflaskapp:latest .
      docker push myacr.azurecr.io/myflaskapp:latest
"""

# 파일 저장
pipeline_file = "azure-pipelines.yml"

with open(pipeline_file, "w") as f:
    f.write(pipeline_content)

print(f"✅ Azure Pipelines 파일 생성 완료: {pipeline_file}")
print()
print("📄 생성된 파일 내용:")
with open(pipeline_file, "r") as f:
    print(f.read())
print()
print("? 참고:")
print("   1. Azure DevOps 프로젝트에서 Pipeline 생성")
print("   2. 이 파일을 저장소에 커밋하여 사용")
print("   3. AzureServiceConnection은 Azure DevOps에서 설정 필요")

## 4. ArgoCD를 활용한 GitOps 방식 배포
ArgoCD는 쿠버네티스 환경에서 GitOps 방식을 실현하는 대표적인 오픈소스 도구입니다. Git 저장소의 선언적 매니페스트를 기준으로 애플리케이션을 자동으로 배포 및 동기화합니다.

### GitOps란?
- 모든 배포/운영 구성을 Git 저장소에 선언적으로 관리
- 변경 사항이 감지되면 자동으로 쿠버네티스 클러스터에 적용

### ArgoCD 설치 (AKS 클러스터)
```bash
kubectl create namespace argocd
kubectl apply -n argocd -f https://raw.githubusercontent.com/argoproj/argo-cd/stable/manifests/install.yaml
```

### ArgoCD CLI 설치 (로컬)
macOS 기준:
```bash
brew install argocd
```

### ArgoCD 서버 접속 및 초기 로그인
```bash
kubectl port-forward svc/argocd-server -n argocd 8080:443
# 브라우저에서 https://localhost:8080 접속
# 초기 admin 패스워드 확인
kubectl get secret argocd-initial-admin-secret -n argocd -o jsonpath="{.data.password}" | base64 -d
```

### 애플리케이션 등록 및 배포
아래는 예시 manifest 저장소를 등록하고 배포하는 방법입니다.
```bash
argocd login localhost:8080
argocd app create myapp \
  --repo https://github.com/your-org/your-repo.git \
  --path k8s-manifests \
  --dest-server https://kubernetes.default.svc \
  --dest-namespace default
argocd app sync myapp
```

### 참고
- 실제 배포 환경에서는 인증, RBAC, 외부 도메인 설정 등 추가 구성이 필요할 수 있습니다.
- 자세한 내용은 [ArgoCD 공식 문서](https://argo-cd.readthedocs.io/)를 참고하세요.

## 5. 정리

이번 실습에서는 GitHub Actions, Azure Pipelines, ArgoCD를 활용해 컨테이너 이미지 빌드부터 ACR 푸시, AKS 배포, GitOps 기반 선언적 배포까지 CI/CD 자동화 전 과정을 GitHub Codespaces 환경에서 실습했습니다.

### 실습한 내용 요약
- ✅ CI/CD 개념 이해
- ✅ GitHub Actions 워크플로우 구성
- ✅ Azure Pipelines 파이프라인 구성
- ✅ ArgoCD를 활용한 GitOps 배포
- ✅ 자동화된 빌드, 테스트, 배포 파이프라인 구축

### 주요 학습 포인트
- **GitHub Actions**: 코드 변경 시 자동으로 빌드 및 ACR 푸시
- **Azure Pipelines**: Azure DevOps를 통한 엔터프라이즈급 CI/CD
- **ArgoCD**: Git 저장소 기반 선언적 배포 및 동기화

이제 컨테이너 이미지를 자동으로 빌드하고, ACR로 푸시하며, AKS에 배포하는 전체 자동화 파이프라인과 GitOps 기반 배포까지 구축할 수 있습니다.

---

## 🎉 실습 완료!

축하합니다! AKS Mini Labs의 모든 실습을 완료했습니다.

- 컨테이너 기본 개념부터 실제 운영 환경 구축까지
- Azure의 관리형 서비스를 활용한 효율적인 클라우드 네이티브 애플리케이션 개발
- CI/CD 자동화를 통한 DevOps 실무 역량 향상

> **GitHub Codespaces** 덕분에 로컬 환경 설정 없이 모든 실습을 진행할 수 있었습니다!